# Imports


In [ ]:
import re
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import time
import math
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df_train = pd.read_csv('/content/drive/MyDrive/Laptop data/4-1/SoP/Lang_Detection/NewData/archive/Language Detection.csv')
df_train = pd.read_csv('/content/drive/MyDrive/Language_Recognition/Language Detection.csv')
df_train.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


# getting only latin derived

In [ ]:
latin_derived = {"English", "French", "Spanish", "Portugeese", "Italian", "Russian", "Sweedish", "Dutch", "Turkish", "German", "Danish"}

df_new = pd.DataFrame(columns = {"Language", "Text"})
df = df_train
for i in range(df.shape[0]):
    if(df.at[i, 'Language'] in latin_derived):
        df_new.loc[len(df_new.index)] = [df.at[i, 'Language'], df.at[i, 'Text']]

df_new.head()

,Language,Text
0,English,"Nature, in the broadest sense, is the natural..."
1,English,"""Nature"" can refer to the phenomena of the phy..."
2,English,"The study of nature is a large, if not the onl..."
3,English,"Although humans are part of nature, human acti..."
4,English,[1] The word nature is borrowed from the Old F...


# Data Pre Processing


In [ ]:
from collections import Counter
c = Counter(df_new['Language'])
print(c)

Counter({'English': 1385, 'French': 1014, 'Spanish': 819, 'Portugeese': 739, 'Italian': 698, 'Russian': 692, 'Sweedish': 676, 'Dutch': 546, 'Turkish': 474, 'German': 470, 'Danish': 428})


In [ ]:
def removeSymbolsAndNumbers(text):        
        text = re.sub(r'[{}]'.format(string.punctuation), '', text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[@]', '', text)
        text = text.replace('"', '')
        text = text.replace('\\', '')
        text = text.replace('\u2009', '')
        text = text.replace('-', '')
        text = text.replace('–', '')
        text = text.replace('—', '')
        text = text.replace('“', '')
        text = text.replace('”', '')
        text = text.replace('•', '')
        text = text.replace('…', '')
        text = text.replace('\u202f', '')
        text = text.replace('№', '')
        text = text.replace('\u200b', '')
        text = text.replace('‑', '')
        text = text.replace('’', '')
        text = text.replace('€', '')
        text = text.replace('„', '')
        text = text.replace('⇒', '')
        text = text.replace('\n', '')
        text = text.replace('\xa0', '')
        text = text.replace('£', '')
        text = text.replace('«', '')
        text = text.replace('°', '')
        text = text.replace('º', '')
        text = text.replace('·', '')
        text = text.replace('»', '')
        text = text.replace('¿', '')
        text = text.replace('×', '')
        text = text.replace('ˈ', '')
        text = text.replace('ˌ', '')
        text = text.replace('ː', '')
        text = text.replace('́', '')
        text = text.replace('̇', '')
        text = text.replace('̇', '')
        text = text.replace( '̇', '')
        text = text.strip()
        return text.lower()

def removeEnglishLetters(text):        
        text = re.sub(r'[a-zA-Z]+', '', text)
        return text.lower()



In [ ]:
df = df_new['Text'].apply(removeSymbolsAndNumbers)
df.head()

0    nature in the broadest sense is the natural ph...
1    nature can refer to the phenomena of the physi...
2    the study of nature is a large if not the only...
3    although humans are part of nature human activ...
4    the word nature is borrowed from the old frenc...
Name: Text, dtype: object

In [ ]:
X = df
y = df_new['Language']

In [ ]:
all_char_list = {' '}

for i in range(len(X)):
  all_char_list = all_char_list.union(set(X[i]))

len(all_char_list)

111

In [ ]:
#Model Creation

#creating all language vectors
#avg num of word per sentences
# 'English': 1385, 
# 'French': 1014, 
# 'Spanish': 819, 
# 'Portugeese': 739, 
# 'Italian': 698, 
# 'Russian': 692, 
# 'Sweedish': 676, 
# 'Dutch': 546, 
# 'Turkish': 474, 
# 'German': 470, 
# 'Danish': 428


X_English = X[y == 'English'].reset_index().drop('index', axis = 1)[0:428]
X_French = X[y == 'French'].reset_index().drop('index', axis = 1)[0:428]
X_Spanish = X[y == 'Spanish'].reset_index().drop('index', axis = 1)[0:428]
X_Portugeese = X[y == 'Portugeese'].reset_index().drop('index', axis = 1)[0:428]
X_Italian = X[y == 'Italian'].reset_index().drop('index', axis = 1)[0:428]
X_Russian = X[y == 'Russian'].reset_index().drop('index', axis = 1)[0:428]
X_Sweedish = X[y == 'Sweedish'].reset_index().drop('index', axis = 1)[0:428]
X_Dutch = X[y == 'Dutch'].reset_index().drop('index', axis = 1)[0:428]
X_Turkish = X[y == 'Turkish'].reset_index().drop('index', axis = 1)[0:428]
X_German = X[y == 'German'].reset_index().drop('index', axis = 1)[0:428]
X_Danish = X[y == 'Danish'].reset_index().drop('index', axis = 1)[0:428]

Data = [X_English, X_French, X_Spanish, X_Portugeese, X_Italian, X_Russian, X_Sweedish, X_Dutch, X_Turkish, X_German, X_Danish]

# Train-Test Split

In [ ]:
trainData = []
testData = []

train_test_split = 0.7

for i in range(len(Data)):
  len_lang = len(Data[i])
  len_train = math.floor(0.7*len_lang)
  trainData.append(Data[i][:len_train].dropna().reset_index().drop('index', axis=1))
  testData.append(Data[i][len_train:].dropna().reset_index().drop('index', axis=1))


# Robustness

In [ ]:
new_char_list = list(all_char_list)

In [ ]:
char_dict = {}
for i in range(len(new_char_list)):
  char_dict[new_char_list[i]] = i
len(char_dict)

111

In [ ]:
robust_char_percent = 0.3
robust_sent_percent = 0.3

In [ ]:
def get_random_number(char):
  while(True):
    c = random.randint(0, len(new_char_list)-1)
    if(c != char_dict[char]): 
      return c

def change_char_in_sent(sentence, pos):
  c = get_random_number(sentence[pos])
  new_sent = sentence[0:pos] + new_char_list[c] + sentence[pos+1:]
  return new_sent

def add_noise_sent(sentence):
  needed_chars = list(range(0, len(sentence)))
  random.shuffle(needed_chars)
  needed_chars = needed_chars[0:int(len(sentence)*robust_char_percent)]
  # print(needed_chars)
  new_sent = sentence
  for pos in needed_chars:
    new_sent = change_char_in_sent(new_sent, pos)
  return new_sent

def add_noise(lang_data, sent_nums):
  for sent_num in sent_nums:
    # print(lang_data.loc[sent_num]['Text'])
    lang_data.loc[sent_num]['Text'] = add_noise_sent(lang_data.loc[sent_num]['Text'])
    # print(lang_data.loc[sent_num]['Text'])
  return lang_data

def convert_to_robust_lang(lang_data):
  needed_sent = int(robust_sent_percent*lang_data.shape[0])
  sent_nums = list(range(0, lang_data.shape[0]))
  random.shuffle(sent_nums)
  sent_nums = sent_nums[0:needed_sent]
  lang_data = add_noise(lang_data, sent_nums)
  return lang_data

def convert_to_robust_data(data):
  newData = []
  for i in range(len(data)):
    newData.append(convert_to_robust_lang(data[i]))
  return newData

In [ ]:
trainRData = []
for i in range(len(trainData)):
  trainRData.append(trainData[i].copy())
trainRData = convert_to_robust_data(trainRData)

In [ ]:
trainRData[0]

,Text
0,nature in the broadest sense is the natural ph...
1,nature can refer to the phenomena of the physi...
2,the study of nature is a large if not the only...
3,although humans are part of nature human activ...
4,the word nature is borrowed from the old frenc...
...,...
294,in the same interview wales also claimed the n...
295,a articшe ćßtled ëhe îeоline ofêwikypediύ in ...
296,the article revealed that since wikipedia had...
297,in july the atlantic reported that the number...


In [ ]:
trainData[0]

,Text
0,nature in the broadest sense is the natural ph...
1,nature can refer to the phenomena of the physi...
2,the study of nature is a large if not the only...
3,although humans are part of nature human activ...
4,the word nature is borrowed from the old frenc...
...,...
294,in the same interview wales also claimed the n...
295,a article titled the decline of wikipedia in ...
296,the article revealed that since wikipedia had...
297,in july the atlantic reported that the number...


# Training

In [ ]:
def generateLetterMappings(dim):
  letter_mappings = {}

  for c in all_char_list:
    s = np.array([])
    for i in range(dim):
      if(random.choice('01') == '1'): s = np.append([1], s)
      else: s = np.append([0], s)
    letter_mappings[c] = s.astype(int)
  
  return letter_mappings

In [ ]:
def getLangVect(dim, N_gram, trainData, letter_mappings):
  def mul(a, b):
    c = a^b
    return 1-c

  def transform(a, n=1):
    arr_a = np.concatenate((a[n:], a[:n]), axis = 0)
    return arr_a

  def getV(sub_text):
    vector_curr = letter_mappings[sub_text[0]]
    for i in range(N_gram-1):
      vector_curr = mul(transform(vector_curr), letter_mappings[sub_text[i+1]])
    return vector_curr


  def getTextVector(texts):
    n = len(texts)
    all_n_gram_vectors = []
    sub_text_first = texts[:N_gram]
    curr_vec = getV(sub_text_first)
    # print(curr_vec)
    all_n_gram_vectors.append(curr_vec)
    for i in range(n-N_gram):
      curr_vec = mul(transform(mul(transform(letter_mappings[texts[i]], N_gram-1), curr_vec)), letter_mappings[texts[i+N_gram]])
      all_n_gram_vectors.append(curr_vec)

    all_gram_vectors = np.array(all_n_gram_vectors)
    # print("All gram: ", all_gram_vectors)
    res = np.sum(all_gram_vectors, axis = 0)/all_gram_vectors.shape[0]
    # print("res: ", res)
    res = np.round_(res)
    # print("rounded res: ", res)
    res = res.astype(int)
    # print("int converted", res)
    return res

  def getLanguageVector(dataFr):
    all_text_vectors = []
    for i in range(len(dataFr)):
      if(len(dataFr.at[i, 'Text'][:N_gram]) >= N_gram):
        all_text_vectors.append(getTextVector(dataFr['Text'][i]))
    
    all_text_vecs = np.array(all_text_vectors)
    res = np.sum(all_text_vecs, axis = 0)/all_text_vecs.shape[0]
    res = np.round_(res)
    res = res.astype(int)
    return res
  
  language_vectors = []
  #latin_derived = {"English", "French", "Spanish", "Portugeese", "Italian", "Russian", "Sweedish", "Dutch", "Turkish", "German", "Danish"}
  start_time = time.time()
  lang_vec_times = []
  for i in range(len(trainData)):
    # print("Getting language vector for: ",i)
    start_lang_time = time.time()
    language_vectors.append(getLanguageVector(trainData[i]))
    lang_vec_times.append(time.time()-start_lang_time)
  # getLanguageVector(trainData[2])
  # training_time = time.time()-start_time
  
  # print("Training time for each lang: ")
  # print(lang_vec_times)
  # print("Training time: ", training_time)
  # print("=========================")
  return language_vectors

# Test

In [ ]:
def test(dim, N_gram, testData, language_vectors, df_given):
  def mul(a, b):
    c = a^b
    return 1-c

  def transform(a, n=1):
    arr_a = np.concatenate((a[n:], a[:n]), axis = 0)
    return arr_a

  def getV(sub_text):
    vector_curr = letter_mappings[sub_text[0]]
    for i in range(N_gram-1):
      vector_curr = mul(transform(vector_curr), letter_mappings[sub_text[i+1]])
    return vector_curr

  def getTextVector(texts):
    n = len(texts)
    if(n < N_gram): return letter_mappings[' ']
    all_n_gram_vectors = []
    sub_text_first = texts[:N_gram]
    
    curr_vec = getV(sub_text_first)
    all_n_gram_vectors.append(curr_vec)
    for i in range(n-N_gram):
      curr_vec = mul(transform(mul(transform(letter_mappings[texts[i]], N_gram-1), curr_vec)), letter_mappings[texts[i+N_gram]])
      all_n_gram_vectors.append(curr_vec)

    all_gram_vectors = np.array(all_n_gram_vectors)
    res = np.sum(all_gram_vectors, axis = 0)/all_gram_vectors.shape[0]
    res = np.round_(res)
    res = res.astype(int)
    return res

  def getDist(a, b):
    d = (a == b)
    unique_elements, counts_elements = np.unique(d, return_counts=True)
    # print(np.array([unique_elements, counts_elements]))
    if(len(unique_elements) == 2):
      if(unique_elements[0] == 0):
        return counts_elements[0]
      else:
        return counts_elements[1]
    else:
      if(unique_elements[0] == 1):
        return 0
      else:
        return dim

  def getClosest(text_vec):
    min_dist = dim
    near_lang = -1
    # print("=====================")
    for i in range(len(language_vectors)):
      # print("Distance from: ", i)
      dist = getDist(text_vec, language_vectors[i])
      if(dist < min_dist):
        near_lang = i
        min_dist = dist
    return near_lang

  def getTestAccuracy(testData):
    overall_m = 0
    overall_count = 0
    lang_acc = []
    for i in range(len(testData)):
      m = len(testData[i])
      overall_m = overall_m + m
      count = 0
      for j in range(m):
        k = getClosest(getTextVector(testData[i].at[j, 'Text']))
        if(i == k):
          count= count +1
        confusion_matrix.iloc[i][k] = confusion_matrix.iloc[i][k] + 1
      overall_count =overall_count + count
      lang_acc.append((count*100)/m)
    lang_acc.append((overall_count*100)/overall_m)
    return lang_acc

  start_test_time = time.time()
  lang_acc = getTestAccuracy(testData)
  # print("Testing time: ")
  # print(time.time() - start_test_time)
  # print(lang_acc)
  df_given.loc[str(dim)] = lang_acc
  return df_given

# Driver Code

In [ ]:
dimensions = []

for i in range(5000, 11000, 1000):
  dimensions.append(i)

N_grams = [3, 4, 5]
# dimensions

In [ ]:
list_of_langs = ['X_English', 'X_French', 
                                 'X_Spanish', 'X_Portugeese', 
                                 'X_Italian', 'X_Russian', 
                                 'X_Sweedish', 'X_Dutch',
                                 'X_Turkish', 'X_German', 
                                 'X_Danish', 'Total']

df_res = pd.DataFrame(columns = list_of_langs)

df_res_rob = pd.DataFrame(columns = list_of_langs)

In [ ]:
confusion_matrix = pd.DataFrame(0, index = list_of_langs[0:len(list_of_langs)-1] ,columns = list_of_langs[0:len(list_of_langs)-1])
confusion_matrix

,X_English,X_French,X_Spanish,X_Portugeese,X_Italian,X_Russian,X_Sweedish,X_Dutch,X_Turkish,X_German,X_Danish
X_English,0,0,0,0,0,0,0,0,0,0,0
X_French,0,0,0,0,0,0,0,0,0,0,0
X_Spanish,0,0,0,0,0,0,0,0,0,0,0
X_Portugeese,0,0,0,0,0,0,0,0,0,0,0
X_Italian,0,0,0,0,0,0,0,0,0,0,0
X_Russian,0,0,0,0,0,0,0,0,0,0,0
X_Sweedish,0,0,0,0,0,0,0,0,0,0,0
X_Dutch,0,0,0,0,0,0,0,0,0,0,0
X_Turkish,0,0,0,0,0,0,0,0,0,0,0
X_German,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_res

,X_English,X_French,X_Spanish,X_Portugeese,X_Italian,X_Russian,X_Sweedish,X_Dutch,X_Turkish,X_German,X_Danish,Total


In [ ]:
dim = 5000
letter_mappings = generateLetterMappings(dim)
N_gram = 3
language_vectors_norm = getLangVect(dim, N_gram, trainData, letter_mappings)
df_res = test(dim, N_gram, testData, language_vectors_norm, df_res)

In [ ]:
language_vectors_norm[20]

IndexError: ignored

In [ ]:
# # For training with (Data + Noise)
# for i in range(len(dimensions)):
#   dim = dimensions[i]
#   letter_mappings = generateLetterMappings(dim)
#   # print(letter_mappings)
#   # print("========================================================================================================================================")
#   print("Dimension: ", dim)
#   N_gram = 3
#   language_vectors_norm = getLangVect(dim, N_gram, trainData, letter_mappings)
#   # language_vectors_rob = getLangVect(dim, N_gram, trainRData, letter_mappings)
#   # print(language_vectors)
#   df_res = test(dim, N_gram, testData, language_vectors_norm, df_res)
#   # df_res_rob = test(dim, N_gram, testData, language_vectors_rob, df_res_rob)
#   # print(df_res_rob-df_res)

In [ ]:
# noiseInLV = 0.05

# def flip(bit):
#   if(bit == 0):
#     return 1
#   else:
#     return 0

# def addNoise(lvs):
#   posi = list(range(0, dim))
#   for i in range(len(lvs)):
#     lvec = lvs[i]
#     # print(lvs[i])
#     random.shuffle(posi)
#     pos_to_flip = posi[0:int(noiseInLV*dim)]
#     for pos in pos_to_flip:
#       lvec[pos] = flip(lvec[pos])
#     lvs[i] = lvec
#     # print(lvs[i])
#   return lvs

In [ ]:
# dim = 50
# letter_mappings = generateLetterMappings(dim)
# N_gram = 3
# language_vectors_norm = getLangVect(dim, N_gram, trainData, letter_mappings)
# language_vectors_rob = np.copy(language_vectors_norm)
# language_vectors_rob = addNoise(language_vectors_rob)

In [ ]:
# For Robustness testing when Language vectors bits are changed by p%

# for i in range(len(dimensions)):
#   dim = dimensions[i]
#   letter_mappings = generateLetterMappings(dim)
#   print("Dimension: ", dim)
#   N_gram = 3
#   language_vectors_norm = getLangVect(dim, N_gram, trainData, letter_mappings)
#   language_vectors_rob = np.copy(language_vectors_norm)
#   language_vectors_rob = addNoise(language_vectors_rob)
#   print(np.unique(language_vectors_norm[0] - language_vectors_rob[0]))
#   # print(language_vectors)
#   df_res = test(dim, N_gram, testData, language_vectors_norm, df_res)
#   df_res_rob = test(dim, N_gram, testData, language_vectors_rob, df_res_rob)
#   print(df_res - df_res_rob)

# Plots


Confusion Matrix

In [ ]:
confusion_matrix.style.background_gradient(cmap ='gnuplot2')\
        .set_properties(**{'font-size': '20px'})

,X_English,X_French,X_Spanish,X_Portugeese,X_Italian,X_Russian,X_Sweedish,X_Dutch,X_Turkish,X_German,X_Danish
X_English,113,1,3,0,5,0,5,0,0,1,1
X_French,0,124,1,2,2,0,0,0,0,0,0
X_Spanish,0,1,122,4,1,0,0,1,0,0,0
X_Portugeese,0,1,0,126,2,0,0,0,0,0,0
X_Italian,1,1,1,0,122,0,0,3,0,0,1
X_Russian,0,0,0,0,0,124,0,0,2,2,1
X_Sweedish,1,0,0,1,1,0,86,2,1,0,37
X_Dutch,2,0,0,0,1,0,0,108,2,7,9
X_Turkish,0,3,2,0,1,0,0,6,106,2,9
X_German,1,1,0,0,3,0,0,4,5,111,4


In [ ]:
df_res['Dimension'] = dimensions
df_res_rob['Dimension'] = dimensions

ValueError: ignored

In [ ]:
df_res

In [ ]:
df_res_rob

In [ ]:
df_new = df_res_rob - df_res
df_new

In [ ]:
ax = plt.gca() 
  
# line plot for math marks
# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_English',
#         color = 'green',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_French',
#         color = 'yellow',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Russian',
#         color = 'red',ax = ax)

df_res.plot(kind = 'line',
        x = 'Dimension',
        y = 'Total',
        color = 'green',ax = ax)

df_res_rob.plot(kind = 'line',
        x = 'Dimension',
        y = 'Total',
        color = 'black',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Dutch',
#         color = 'brown',ax = ax)


# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Italian',
#         color = 'violet',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Sweedish',
#         color = 'blue',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Turkish',
#         color = 'purple',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Danish',
#         color = 'indigo',ax = ax)

# df_res.plot(kind = 'line',
#         x = 'Dimension',
#         y = 'X_Spanish',
#         color = 'cyan',ax = ax)
  
# show the plot
f = plt.figure()
f.set_figwidth(40000)
f.set_figheight(100)
plt.show()

In [ ]:
  # #{"English", "French", "Spanish", "Portugeese", "Italian", "Russian", "Sweedish", "Dutch", "Turkish", "German", "Danish"}
  # #Counter(
  #     {'English': 1385, 
  #      'French': 1014, 
  #      'Spanish': 819, 
  #      'Portugeese': 739, 
  #      'Italian': 698, 
  #      'Russian': 692, 
  #      'Sweedish': 676, 
  #      'Dutch': 546, 
  #      'Turkish': 474, 
  #      'German': 470, 
  #      'Danish': 428}